In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
pd.set_option("Max_Columns",999)

In [48]:
import math
from tqdm import tqdm
tqdm.pandas()

In [2]:
import plotly.express as px
import plotly.graph_objects as go
import chart_studio.plotly as py
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import cufflinks as cf

init_notebook_mode(connected=True)
cf.go_offline()


In [17]:
df=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [8]:
df.head()

,ID,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Action
0,59276,49418.0,27015.0,54748.0,NaN,78.0,78.0,0.0,1.0,29.0,1.0,0.0,allow
1,18986,58120.0,25174.0,NaN,0.0,66.0,66.0,0.0,NaN,0.0,1.0,0.0,deny
2,11131,46751.0,53.0,46751.0,53.0,220.0,90.0,130.0,2.0,30.0,NaN,1.0,allow
3,57352,52596.0,NaN,0.0,0.0,66.0,66.0,0.0,1.0,NaN,1.0,0.0,drop
4,17229,50584.0,6881.0,29773.0,NaN,502.0,150.0,352.0,2.0,1200.0,NaN,1.0,allow


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45870 entries, 0 to 45869
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    45870 non-null  int64  
 1   Source Port           36683 non-null  float64
 2   Destination Port      36693 non-null  float64
 3   NAT Source Port       36756 non-null  float64
 4   NAT Destination Port  36674 non-null  float64
 5   Bytes                 36607 non-null  float64
 6   Bytes Sent            36764 non-null  float64
 7   Bytes Received        36693 non-null  float64
 8   Packets               36818 non-null  float64
 9   Elapsed Time (sec)    36584 non-null  float64
 10  pkts_sent             36667 non-null  float64
 11  pkts_received         36711 non-null  float64
 12  Action                45870 non-null  object 
dtypes: float64(11), int64(1), object(1)
memory usage: 4.5+ MB


In [27]:
9187/45870

0.2002834096359276

In [10]:
df.isnull().sum()


ID                         0
Source Port             9187
Destination Port        9177
NAT Source Port         9114
NAT Destination Port    9196
Bytes                   9263
Bytes Sent              9106
Bytes Received          9177
Packets                 9052
Elapsed Time (sec)      9286
pkts_sent               9203
pkts_received           9159
Action                     0
dtype: int64

In [15]:
df["Action"].value_counts(normalize=True)
#51 percent is allow 
# May be undersampling of allow 22 % . Reset both is very small(If Undersampling maybe smote of reset both)

allow         0.574406
deny          0.228690
drop          0.196098
reset-both    0.000807
Name: Action, dtype: float64

In [16]:
df

,ID,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Action
0,59276,49418.0,27015.0,54748.0,NaN,78.0,78.0,0.0,1.0,29.0,1.0,0.0,allow
1,18986,58120.0,25174.0,NaN,0.0,66.0,66.0,0.0,NaN,0.0,1.0,0.0,deny
2,11131,46751.0,53.0,46751.0,53.0,220.0,90.0,130.0,2.0,30.0,NaN,1.0,allow
3,57352,52596.0,NaN,0.0,0.0,66.0,66.0,0.0,1.0,NaN,1.0,0.0,drop
4,17229,50584.0,6881.0,29773.0,NaN,502.0,150.0,352.0,2.0,1200.0,NaN,1.0,allow
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45865,36322,NaN,53.0,NaN,53.0,94.0,94.0,0.0,NaN,29.0,NaN,NaN,allow
45866,36418,NaN,53.0,44635.0,53.0,177.0,94.0,83.0,2.0,NaN,NaN,1.0,allow
45867,2289,50438.0,445.0,0.0,0.0,70.0,NaN,0.0,1.0,0.0,1.0,0.0,drop
45868,10229,37785.0,80.0,22917.0,80.0,2293.0,1576.0,NaN,12.0,15.0,6.0,NaN,allow


In [18]:
test

,ID,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,23127,50142.0,445.0,0.0,0.0,NaN,70.0,0.0,1.0,0.0,1.0,NaN
1,61535,49496.0,53.0,NaN,53.0,194.0,NaN,92.0,2.0,30.0,1.0,1.0
2,31457,49209.0,53.0,27106.0,NaN,199.0,102.0,NaN,NaN,31.0,NaN,1.0
3,49294,49225.0,53.0,NaN,53.0,295.0,101.0,194.0,NaN,48.0,2.0,2.0
4,8826,NaN,NaN,0.0,NaN,70.0,70.0,0.0,1.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
19657,1083,NaN,25174.0,0.0,0.0,146.0,146.0,0.0,NaN,NaN,1.0,NaN
19658,20633,NaN,53.0,49003.0,53.0,194.0,102.0,92.0,2.0,31.0,1.0,1.0
19659,50324,27005.0,27015.0,NaN,27015.0,878.0,NaN,752.0,2.0,30.0,1.0,1.0
19660,16271,49676.0,53.0,NaN,NaN,216.0,110.0,106.0,2.0,30.0,1.0,1.0


In [19]:
test ["test"]=1
df["test"]=0
total=pd.concat([df,test],axis=0)

In [20]:
total

,ID,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Action,test
0,59276,49418.0,27015.0,54748.0,NaN,78.0,78.0,0.0,1.0,29.0,1.0,0.0,allow,0
1,18986,58120.0,25174.0,NaN,0.0,66.0,66.0,0.0,NaN,0.0,1.0,0.0,deny,0
2,11131,46751.0,53.0,46751.0,53.0,220.0,90.0,130.0,2.0,30.0,NaN,1.0,allow,0
3,57352,52596.0,NaN,0.0,0.0,66.0,66.0,0.0,1.0,NaN,1.0,0.0,drop,0
4,17229,50584.0,6881.0,29773.0,NaN,502.0,150.0,352.0,2.0,1200.0,NaN,1.0,allow,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19657,1083,NaN,25174.0,0.0,0.0,146.0,146.0,0.0,NaN,NaN,1.0,NaN,NaN,1
19658,20633,NaN,53.0,49003.0,53.0,194.0,102.0,92.0,2.0,31.0,1.0,1.0,NaN,1
19659,50324,27005.0,27015.0,NaN,27015.0,878.0,NaN,752.0,2.0,30.0,1.0,1.0,NaN,1
19660,16271,49676.0,53.0,NaN,NaN,216.0,110.0,106.0,2.0,30.0,1.0,1.0,NaN,1


In [26]:
total.query("test==0")
len(test)

19662

In [25]:
test.isnull().sum()

ID                         0
Source Port             3969
Destination Port        3979
NAT Source Port         4067
NAT Destination Port    3942
Bytes                   3877
Bytes Sent              3778
Bytes Received          3929
Packets                 3926
Elapsed Time (sec)      3959
pkts_sent               3954
pkts_received           3949
test                       0
dtype: int64

In [28]:
4067/20000

0.20335

In [29]:
# 20 % is null

In [56]:
col=['ID','Source Port', 'Destination Port', 'NAT Source Port',
       'NAT Destination Port', 'Bytes', 'Bytes Sent', 'Bytes Received',
       'Packets', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received','Action','test','nancount']

In [53]:
df["nancount"]=0

In [54]:
for i in range(len(df)) :
        df.loc[i,"nancount"]= df.iloc[i].isnull().sum()

In [59]:
df=df[col].copy()

In [60]:
df

,ID,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Action,test,nancount
0,59276,49418.0,27015.0,54748.0,NaN,78.0,78.0,0.0,1.0,29.0,1.0,0.0,allow,0,1
1,18986,58120.0,25174.0,NaN,0.0,66.0,66.0,0.0,NaN,0.0,1.0,0.0,deny,0,2
2,11131,46751.0,53.0,46751.0,53.0,220.0,90.0,130.0,2.0,30.0,NaN,1.0,allow,0,1
3,57352,52596.0,NaN,0.0,0.0,66.0,66.0,0.0,1.0,NaN,1.0,0.0,drop,0,2
4,17229,50584.0,6881.0,29773.0,NaN,502.0,150.0,352.0,2.0,1200.0,NaN,1.0,allow,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45865,36322,NaN,53.0,NaN,53.0,94.0,94.0,0.0,NaN,29.0,NaN,NaN,allow,0,5
45866,36418,NaN,53.0,44635.0,53.0,177.0,94.0,83.0,2.0,NaN,NaN,1.0,allow,0,3
45867,2289,50438.0,445.0,0.0,0.0,70.0,NaN,0.0,1.0,0.0,1.0,0.0,drop,0,1
45868,10229,37785.0,80.0,22917.0,80.0,2293.0,1576.0,NaN,12.0,15.0,6.0,NaN,allow,0,2


In [61]:
df.to_csv("train_with_nancount.csv",index=False)

In [63]:
df["nancount"].value_counts()
#can drop rows with 7,8,9 nan values

2    13583
1    10819
3    10163
4     5021
0     3936
5     1823
6      439
7       77
8        7
9        2
Name: nancount, dtype: int64

In [73]:
df=df[df["nancount"]<7].copy()

In [75]:
# Removed 85 rows

In [77]:
df.to_csv("straightfitxgb.csv",index=False)

In [78]:
df

,ID,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Action,test,nancount
0,59276,49418.0,27015.0,54748.0,NaN,78.0,78.0,0.0,1.0,29.0,1.0,0.0,allow,0,1
1,18986,58120.0,25174.0,NaN,0.0,66.0,66.0,0.0,NaN,0.0,1.0,0.0,deny,0,2
2,11131,46751.0,53.0,46751.0,53.0,220.0,90.0,130.0,2.0,30.0,NaN,1.0,allow,0,1
3,57352,52596.0,NaN,0.0,0.0,66.0,66.0,0.0,1.0,NaN,1.0,0.0,drop,0,2
4,17229,50584.0,6881.0,29773.0,NaN,502.0,150.0,352.0,2.0,1200.0,NaN,1.0,allow,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45865,36322,NaN,53.0,NaN,53.0,94.0,94.0,0.0,NaN,29.0,NaN,NaN,allow,0,5
45866,36418,NaN,53.0,44635.0,53.0,177.0,94.0,83.0,2.0,NaN,NaN,1.0,allow,0,3
45867,2289,50438.0,445.0,0.0,0.0,70.0,NaN,0.0,1.0,0.0,1.0,0.0,drop,0,1
45868,10229,37785.0,80.0,22917.0,80.0,2293.0,1576.0,NaN,12.0,15.0,6.0,NaN,allow,0,2
